In [239]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sp
import sklearn
import datetime

In [240]:
df07 = pd.read_csv("../../data/raw/activaciones_samur_2017.csv",sep=';', encoding='latin-1')
df08 = pd.read_csv("../../data/raw/activaciones_samur_2018.csv",sep=';', encoding='latin-1')
df09 = pd.read_csv("../../data/raw/activaciones_samur_2019.csv",sep=';', encoding='latin-1')
dataframe = df09 

In [241]:
month_dict = {"ENERO":1,"FEBRERO":2,"MARZO":3,"ABRIL":4,"MAYO":5,"JUNIO":6,"JULIO":7,"AGOSTO":8,"SEPTIEMBRE":9,"OCTUBRE":10,"NOVIEMBRE":11,"DICIEMBRE":12} 
year = dataframe["Año"].unique()[0] 

In [242]:
dataframe

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital
0,2019,ENERO,0:08:09,0:08:15,Intoxicación etílica,CENTRO,NaN
1,2019,ENERO,0:09:13,NaN,Violencia de genero,VALLECAS PTE.,NaN
2,2019,ENERO,0:24:12,0:24:22,Otros,CENTRO,NaN
3,2019,ENERO,0:28:55,NaN,Intoxicación etílica,CENTRO,NaN
4,2019,ENERO,0:29:11,0:34:02,"Casual: caída, etc",CENTRO,NaN
...,...,...,...,...,...,...,...
149899,2019,DICIEMBRE,23:28:26,23:35:50,Intoxicación etílica,CHAMARTIN,NaN
149900,2019,DICIEMBRE,23:32:19,23:37:39,Patología digestiva,LATINA,Central de la Defensa
149901,2019,DICIEMBRE,23:35:18,23:38:41,Intoxicación etílica,ARGANZUELA,NaN
149902,2019,DICIEMBRE,NaN,NaN,1 SVB,CENTRO,NaN


In [243]:
def striptimeHandler(x):
    """
    Utility method to handle non valid times when transforming string dates to datetime.datetime objects
    """
    if x == -1:
        return -1
    else:
        return datetime.datetime.strptime(x,"%X")

In [244]:
#Fill n/a of emergencies without destination hospital.
dataframe["Hospital"].fillna("No derivado",inplace=True)
#Fill n/a of emergencies without intervenction time -1.
dataframe["Hora Intervención"].fillna(-1,inplace=True)
#Drop lines without intervention time. 
dataframe.dropna(inplace=True)
#Retype intervention type and reindex using that column 
dataframe["Hora Solicitud"] = dataframe["Hora Solicitud"].apply(lambda x : datetime.datetime.strptime(x,"%X"))
dataframe["Hora Intervención"] = dataframe["Hora Intervención"].apply(lambda x : striptimeHandler(x))
dataframe.reindex()

,Año,Mes,Hora Solicitud,Hora Intervención,Código,Distrito,Hospital
0,2019,ENERO,1900-01-01 00:08:09,1900-01-01 00:08:15,Intoxicación etílica,CENTRO,No derivado
1,2019,ENERO,1900-01-01 00:09:13,-1,Violencia de genero,VALLECAS PTE.,No derivado
2,2019,ENERO,1900-01-01 00:24:12,1900-01-01 00:24:22,Otros,CENTRO,No derivado
3,2019,ENERO,1900-01-01 00:28:55,-1,Intoxicación etílica,CENTRO,No derivado
4,2019,ENERO,1900-01-01 00:29:11,1900-01-01 00:34:02,"Casual: caída, etc",CENTRO,No derivado
...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,1900-01-01 23:12:54,1900-01-01 23:20:13,"Casual: caída, etc",HORTALEZA,Ramón y Cajal
149898,2019,DICIEMBRE,1900-01-01 23:26:55,1900-01-01 23:31:16,Apertura de puerta,CHAMBERI,No derivado
149899,2019,DICIEMBRE,1900-01-01 23:28:26,1900-01-01 23:35:50,Intoxicación etílica,CHAMARTIN,No derivado
149900,2019,DICIEMBRE,1900-01-01 23:32:19,1900-01-01 23:37:39,Patología digestiva,LATINA,Central de la Defensa


In [245]:
dataframe["Devuelto"] = dataframe["Hora Intervención"].apply(lambda x : (x == -1))
#All emergencies without an intervention time are tagged as 'Devuelto'(returned, as canceled)

In [246]:
def encontrar_los_dias(dataframe):
    """
    As dataset has no dates but only hours, that column will be explored and will final 'leaps' to lower hours ie: 23:12:00 -> 01:12:43) and identify that as a day increase.
    returns number of days count.
    """
    temp = dataframe["Hora Solicitud"].to_numpy()
    dias = [0]
    day = 0
    for i in range(0,len(temp)-1):
        dias.append(day)
        if(temp[i] >temp[i+1]):
            day +=1
    return dias

In [247]:
def flatten2D(array):
    """
    Simple flattening algorith to turn 2D array to 1D array
    """
    temp = []
    for i in array:
        for x in i:
            temp.append(x)
    return temp

In [248]:
# Flattening method is applied to dataset, month by month to avoid errors.
dias_desde_inicio = []
for mes in dataframe["Mes"].unique():
    dias_desde_inicio.append(encontrar_los_dias(dataframe[dataframe["Mes"] == mes]))
dias_desde_inicio = flatten2D(dias_desde_inicio)
dataframe["Dias_desde_inicio"] = dias_desde_inicio

In [249]:
#Max month day is 30 instead of 31 due to zero-based numbering for month day.
dataframe.describe()

,Año,Dias_desde_inicio
count,146780.0,146780.000000
mean,2019.0,14.682736
std,0.0,8.882540
min,2019.0,0.000000
25%,2019.0,7.000000
50%,2019.0,15.000000
75%,2019.0,22.000000
max,2019.0,30.000000


In [250]:
#Se construye la fecha de cada solicitud, incluyendo el año del dataset y el mes de la solicitud.
#Se suma uno al dia para corregir la desviación causada por encontrar el dia.
#La hora, minuto y segundo se encuentran extrayéndolas del datetime ya creado previamente.
#Un nuevo objeto datetime se crea con estos datos y se añade a ua lista.
#El proceso se aplica tanto a la solicitud como a la intervención, con la diferencia de que se evita el valor -1, ya que eso significa que esa hora no existe.
fecha_solicitud = []
fecha_intervención = []
for x in dataframe.iterrows():
    year = x[1][0]
    month = month_dict.get(x[1][1])
    day = x[1][8] + 1
    hour = x[1][2].hour
    minute = x[1][2].minute
    second = x[1][2].second
    temp = datetime.datetime(year, month,day,hour,minute,second)
    fecha_solicitud.append(temp)
for x in dataframe.iterrows():
    if x[1][7]: 
        fecha_intervención.append(-1)
    else:
        year = x[1][0]
        month = month_dict.get(x[1][1])
        day = x[1][8] + 1
        hour = x[1][3].hour
        minute = x[1][3].minute
        second = x[1][3].second
        temp = datetime.datetime(year, month,day,hour,minute,second)
        fecha_intervención.append(temp)

In [251]:
#List are added to dataframe and redundant columns are dropped.
dataframe["Solicitud"] = fecha_solicitud
dataframe["Intervención"] = fecha_intervención
dataframe.drop(["Hora Solicitud","Hora Intervención","Dias_desde_inicio"],axis=1,inplace=True)

In [252]:
#At last, week day column is created for emergency (0 to 6 numbering, monday starting week)
dataframe["Dia de la semana"] = dataframe["Solicitud"].apply(lambda x: x.weekday())

In [253]:
#Final resulting dataframe
dataframe

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana
0,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,False,2019-01-01 00:08:09,2019-01-01 00:08:15,1
1,2019,ENERO,Violencia de genero,VALLECAS PTE.,No derivado,True,2019-01-01 00:09:13,-1,1
2,2019,ENERO,Otros,CENTRO,No derivado,False,2019-01-01 00:24:12,2019-01-01 00:24:22,1
3,2019,ENERO,Intoxicación etílica,CENTRO,No derivado,True,2019-01-01 00:28:55,-1,1
4,2019,ENERO,"Casual: caída, etc",CENTRO,No derivado,False,2019-01-01 00:29:11,2019-01-01 00:34:02,1
...,...,...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,"Casual: caída, etc",HORTALEZA,Ramón y Cajal,False,2019-12-31 23:12:54,2019-12-31 23:20:13,1
149898,2019,DICIEMBRE,Apertura de puerta,CHAMBERI,No derivado,False,2019-12-31 23:26:55,2019-12-31 23:31:16,1
149899,2019,DICIEMBRE,Intoxicación etílica,CHAMARTIN,No derivado,False,2019-12-31 23:28:26,2019-12-31 23:35:50,1
149900,2019,DICIEMBRE,Patología digestiva,LATINA,Central de la Defensa,False,2019-12-31 23:32:19,2019-12-31 23:37:39,1


In [254]:
dataframe[dataframe["Mes"] == "AGOSTO"]

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana
88050,2019,AGOSTO,Agresión sin especificar,LATINA,Doce de Octubre,False,2019-08-01 00:09:35,2019-08-01 00:18:04,3
88051,2019,AGOSTO,Intoxicación etílica,CENTRO,No derivado,False,2019-08-01 00:12:22,2019-08-01 00:20:47,3
88052,2019,AGOSTO,Servicios de seguimiento de riesgos,MONCLOA,No derivado,False,2019-08-01 00:20:12,2019-08-01 00:25:09,3
88053,2019,AGOSTO,Patología anafiláctica,ARGANZUELA,No derivado,False,2019-08-01 00:38:17,2019-08-01 00:47:17,3
88054,2019,AGOSTO,Patología psicosomática,CENTRO,No derivado,False,2019-08-01 00:50:39,2019-08-01 00:55:43,3
...,...,...,...,...,...,...,...,...,...
98091,2019,AGOSTO,Intoxicación etílica,VALLECAS PTE.,INFANTA LEONOR,False,2019-08-31 23:38:15,2019-08-31 23:41:43,5
98092,2019,AGOSTO,Agresión sin especificar,CENTRO,No derivado,False,2019-08-31 23:39:02,2019-08-31 23:48:21,5
98093,2019,AGOSTO,Otros,CENTRO,No derivado,False,2019-08-31 23:40:47,2019-08-31 23:48:45,5
98094,2019,AGOSTO,Autolisis no traumática,MORATALAZ,Gregorio Marañón,False,2019-08-31 23:56:12,2019-08-31 00:07:57,5


In [255]:
dataframe[dataframe["Mes"] == "DICIEMBRE"]

,Año,Mes,Código,Distrito,Hospital,Devuelto,Solicitud,Intervención,Dia de la semana
136399,2019,DICIEMBRE,Otros,CENTRO,Ramón y Cajal,False,2019-12-01 00:01:41,2019-12-01 00:02:17,6
136401,2019,DICIEMBRE,Patología cardiovascular,CENTRO,No derivado,False,2019-12-01 00:07:09,2019-12-01 00:11:40,6
136402,2019,DICIEMBRE,Patología cardiovascular,VALLECAS PTE.,No derivado,False,2019-12-01 00:09:28,2019-12-01 00:14:46,6
136403,2019,DICIEMBRE,Patología cardiovascular,CHAMARTIN,No derivado,False,2019-12-01 00:09:56,2019-12-01 00:18:01,6
136404,2019,DICIEMBRE,Patología psicosomática,VICALVARO,No derivado,True,2019-12-01 00:13:55,-1,6
...,...,...,...,...,...,...,...,...,...
149897,2019,DICIEMBRE,"Casual: caída, etc",HORTALEZA,Ramón y Cajal,False,2019-12-31 23:12:54,2019-12-31 23:20:13,1
149898,2019,DICIEMBRE,Apertura de puerta,CHAMBERI,No derivado,False,2019-12-31 23:26:55,2019-12-31 23:31:16,1
149899,2019,DICIEMBRE,Intoxicación etílica,CHAMARTIN,No derivado,False,2019-12-31 23:28:26,2019-12-31 23:35:50,1
149900,2019,DICIEMBRE,Patología digestiva,LATINA,Central de la Defensa,False,2019-12-31 23:32:19,2019-12-31 23:37:39,1


In [256]:
# As it can be seen ,there are some missing dates in dataframe, and that will result in a shipt on weekly repeating patterns, like ethilic intoxication related emergencies. Using that pattern, missing days will bve identified and inserted to fill the gaps. That will be covered in notebook 'SAMUR Dataset study'

In [257]:
#Para su facil manejo, se exporta a .csv
dataframe.to_csv("../../data/processed/Dataset_SAMUR_2019.csv")